<a href="https://colab.research.google.com/github/knolasco/Microsoft_Object_Detection/blob/main/Microsoft_Surfer_Detection_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

I will use google colab to perform transfer learning because of the free GPU (it would take too long to run on my CPU). From this notebook, I will download the latest checkpoint and apply it to a .py file elsewhere.

# Install important packages

In [1]:
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install -U --pre tensorflow=="2.*"
!pip install tf_slim
!pip install pycocotools
import os
import pathlib

Requirement already up-to-date: tensorflow==2.* in /usr/local/lib/python3.7/dist-packages (2.4.1)
     |████████████████████████████████| 358kB 8.6MB/s 


# Make folders for the project

In [4]:
%%bash
mkdir Tensorflow
cd Tensorflow/
mkdir scripts
mkdir workspace
cd workspace
mkdir models
mkdir images
mkdir annotations
mkdir pre-trained-models
cd images
mkdir test
mkdir train

In [5]:
# clone the tensorflow repo
%%bash
cd Tensorflow
git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...


In [6]:
%%bash
cd Tensorflow/models/research/
protoc object_detection/protos/*.proto --python_out=.

In [7]:
%%bash
cd Tensorflow/models/research/
cp object_detection/packages/tf2/setup.py .

In [8]:
%%bash 
cd Tensorflow/models/research 
pip install .

Processing /content/Tensorflow/models/research
  Created wheel for object-detection: filename=object_detection-0.1-cp37-none-any.whl size=1630675 sha256=48053b6f725268a9b9b533e08414866926ce3993fbb3e8f137615a4aceb143d6
  Stored in directory: /tmp/pip-ephem-wheel-cache-pcucvt94/wheels/ac/8a/90/02d8042c0333b38a96341f55d93b379bb170f98b196de6b536
  Created wheel for avro-python3: filename=avro_python3-1.10.2-cp37-none-any.whl size=44011 sha256=d1b331e5314dd21df3f59315adfa280adfea3857c754e5c9a32903380ed6c7f2
  Stored in directory: /root/.cache/pip/wheels/ee/ee/18/c466221ca6900e3efce2f4ea9c329288808679aecdcb2838d3
  Created wheel for dill: filename=dill-0.3.1.1-cp37-none-any.whl size=78532 sha256=6e7681ff4e45a2b3b34718bac945d2e13f873add4423e5cbeb52edb2e063843e
  Stored in directory: /root/.cache/pip/wheels/59/b1/91/f02e76c732915c4015ab4010f3015469866c1eb9b14058d8e7
  Created wheel for future: filename=future-0.18.2-cp37-none-any.whl size=491058 sha256=a7512794d4f34c4f7329a75b05c7363444871d5b0

ERROR: multiprocess 0.70.11.1 has requirement dill>=0.3.3, but you'll have dill 0.3.1.1 which is incompatible.
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.25.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: apache-beam 2.28.0 has requirement avro-python3!=1.9.2,<1.10.0,>=1.8.1, but you'll have avro-python3 1.10.2 which is incompatible.


# Set up paths

In [9]:
WORKSPACE_PATH = 'Tensorflow/workspace'
SCRIPTS_PATH = 'Tensorflow/scripts'
APIMODEL_PATH = 'Tensorflow/models'
ANNOTATION_PATH = WORKSPACE_PATH +'/annotations'
IMAGE_PATH = WORKSPACE_PATH + '/images'
MODEL_PATH = WORKSPACE_PATH + '/models'
PRETRAINED_MODEL_PATH = WORKSPACE_PATH + '/pre-trained-models'
CONFIG_PATH = MODEL_PATH + '/my_ssd_mobnet/pipeline.config'
CHECKPOINT_PATH = MODEL_PATH + '/my_ssd_mobnet/'

In [10]:
# these are the labels for the object detection
labels = [{'name' : 'surfer', 'id': 1},
             {'name': 'zone', 'id' : 2}]

# create the file as pbtxt and save it to the annotation folder
with open(ANNOTATION_PATH + '/label_map.pbtxt', 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

# Create the TF Records

In [11]:
# move the train/test.tar files to the appropriate folders
!cp {'/content/drive/MyDrive/Microsoft_Project/test.tar'} {'/content/Tensorflow/workspace/images'}
!cp {'/content/drive/MyDrive/Microsoft_Project/train.tar'} {'/content/Tensorflow/workspace/images'}


# move the generatetf records file
!cp {'/content/drive/MyDrive/Microsoft_Project/generate_tfrecord.py'} {'/content/Tensorflow/scripts'}

In [12]:
# open the tar files
import tarfile
test_tar = tarfile.open('/content/Tensorflow/workspace/images/test.tar')
test_tar.extractall('/content/Tensorflow/workspace/images')
test_tar.close()

train_tar = tarfile.open('/content/Tensorflow/workspace/images/train.tar')
train_tar.extractall('/content/Tensorflow/workspace/images')
train_tar.close()

# Preprocess the images
The plan is to convert all images to black and white and add a blur. This should all be done before creating the TFRecords

In [13]:
# we will make a class for this
import numpy as np
from PIL import Image, ImageFilter, ImageOps

class Image_Sets():
  def __init__(self, image_path):
    self.image_path = image_path
  
  def process_images(self):
    for image in os.listdir(self.image_path):
      if '.xml' not in image:
        filename = os.path.join(self.image_path, image)
        img = Image.open(filename) # open the image
        img = ImageOps.solarize(img) # inverts pixel values above the 128 threshold
        img = img.convert('LA') # make grayscale
        img = img.convert('RGB')
        img.save(filename) # save it and overwrite


In [14]:
train_images = Image_Sets('/content/Tensorflow/workspace/images/train')
train_images.process_images()

test_images = Image_Sets('/content/Tensorflow/workspace/images/test')
test_images.process_images()

In [15]:
!python /content/Tensorflow/scripts/generate_tfrecord.py -x /content/Tensorflow/workspace/images/train -l /content/Tensorflow/workspace/annotations/label_map.pbtxt -o /content/Tensorflow/workspace/annotations/train.record

!python /content/Tensorflow/scripts/generate_tfrecord.py -x /content/Tensorflow/workspace/images/test -l /content/Tensorflow/workspace/annotations/label_map.pbtxt -o /content/Tensorflow/workspace/annotations/test.record

Successfully created the TFRecord file: /content/Tensorflow/workspace/annotations/train.record
Successfully created the TFRecord file: /content/Tensorflow/workspace/annotations/test.record


# Copy the model config to the training folder

In [16]:
CUSTOM_MODEL_NAME = 'my_ssd_mobnet'

In [17]:
%%bash
cd Tensorflow/workspace/pre-trained-models/
mkdir checkpoint
mkdir saved_model
cd saved_model
mkdir variables

In [18]:
# move the checkpoints
!cp {'/content/drive/MyDrive/Microsoft_Project/checkpoint'} {'/content/Tensorflow/workspace/pre-trained-models/checkpoint'}
!cp {'/content/drive/MyDrive/Microsoft_Project/ckpt-0.data-00000-of-00001'} {'/content/Tensorflow/workspace/pre-trained-models/checkpoint'}
!cp {'/content/drive/MyDrive/Microsoft_Project/ckpt-0.index'} {'/content/Tensorflow/workspace/pre-trained-models/checkpoint'}

# move the variables
!cp {'/content/drive/MyDrive/Microsoft_Project/variables.data-00000-of-00001'} {'/content/Tensorflow/workspace/pre-trained-models/saved_model/variables'}
!cp {'/content/drive/MyDrive/Microsoft_Project/variables.index'} {'/content/Tensorflow/workspace/pre-trained-models/saved_model/variables'}

# move the saved model
!cp {'/content/drive/MyDrive/Microsoft_Project/saved_model.pb'} {'/content/Tensorflow/workspace/pre-trained-models/saved_model'}

# move the config file
!cp {'/content/drive/MyDrive/Microsoft_Project/pipeline.config'} {'/content/Tensorflow/workspace/pre-trained-models'}

In [19]:
# upload the pipeline.config, checkpoints and variables from the pretrained model
# before running the code below
!mkdir {'Tensorflow/workspace/models/' + CUSTOM_MODEL_NAME}
!cp {PRETRAINED_MODEL_PATH + '/pipeline.config'} {MODEL_PATH + '/' + CUSTOM_MODEL_NAME}

# Update the config file for transfer learning

In [20]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [21]:
CONFIG_PATH = MODEL_PATH + '/' + CUSTOM_MODEL_NAME + '/pipeline.config'
config = config_util.get_configs_from_pipeline_file(CONFIG_PATH)

In [22]:
# create a shell for the pipeline config
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(CONFIG_PATH, 'r') as f:
    proto_str = f.read()
    text_format.Merge(proto_str, pipeline_config)

In [23]:

# edit the config to work for our project
pipeline_config.model.ssd.num_classes = 2
pipeline_config.train_config.batch_size = 64
pipeline_config.train_config.fine_tune_checkpoint = PRETRAINED_MODEL_PATH + '/checkpoint/ckpt-0' # edit this checkpoint in case I stop training early
pipeline_config.train_config.fine_tune_checkpoint_type = 'detection'
pipeline_config.train_input_reader.label_map_path = ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/train.record']
pipeline_config.eval_input_reader[0].label_map_path = ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/test.record']

In [24]:
# input this pipeline_config into our project directory
config_text = text_format.MessageToString(pipeline_config)
with tf.io.gfile.GFile(CONFIG_PATH, 'wb') as f:
    f.write(config_text)

# Train the model

In [25]:
with tf.device('/device:GPU:0'):
  !python /content/Tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=/content/Tensorflow/workspace/models/my_ssd_mobnet --pipeline_config_path=/content/Tensorflow/workspace/models/my_ssd_mobnet/pipeline.config --num_train_steps=10000

2021-03-30 20:42:56.145936: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-03-30 20:42:59.166373: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-03-30 20:42:59.167219: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-03-30 20:42:59.180210: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-03-30 20:42:59.180754: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.75GiB deviceMemoryBandwidth: 298.08GiB/s
2021-03-30 20:42:59.180789: I tensorflow/stream_executor/platform/default/dso_loade